<a href="https://colab.research.google.com/github/pamunarr/Chroma/blob/main/Chroma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chroma - Caso de uso

**ARDA II - NoSQL**

*Andra-Iulia Cebuc, Pablo Munarriz Senosiain*

En este cuaderno mostramos un caso de uso del SGBD vectorial [Chroma](https://docs.trychroma.com/). Además de poner en práctica lo aprendido, nuestro objetivo es relacionarlo con la asignatura Aprendizaje Profundo, de forma que podamos ver el funcionamiento de Chroma en el contexto para el que fue diseñado.

## Librerías

En primer lugar, debemos instalar las librerías necesarias. Para usar Chroma basta con instalar `chromadb`, pero también utilizaremos `datasets` y `cohere`.

La librería `datasets` nos sirve para descargar conjuntos de datos desde HuggingFace. La librería `cohere` nos va a permitir usar una función de embedding concreta.

Una vez instaladas, reiniciamos el kernel para vaciar la memoria RAM.

In [ ]:
!pip install chromadb
!pip install datasets -Uqq
!pip install -U cohere

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
import chromadb
from datasets import load_dataset
import cohere

# Dataset

Vamos a trabajar con datos de la Wikipedia, concretamente, vamos a descargarnos documentos en euskera y gallego. Para ello recurrimos a HuggingFace, en particular a [este dataset](https://huggingface.co/datasets/Cohere/wikipedia-2023-11-embed-multilingual-v3), que contiene una copia de Wikipedia del 01/11/2023 en más de 300 idiomas. Empezamos descargando los documentos en euskera.

In [2]:
wikipedia_eu = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "eu", split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Vamos a echar un vistazo al esquema de los datos.

In [ ]:
wikipedia_eu

Dataset({
    features: ['_id', 'url', 'title', 'text', 'emb'],
    num_rows: 1327579
})

Tenemos cinco características:
 - `_id`: un identificador,
 - `url`: dirección URL del documento,
 - `title`: título del documento,
 - `text`: contenido del documento,
 - `emb`: un embedding del documento.

Ya que, para cada documento, disponemos de la URL y de un embedding, vamos a aprovechar para no guardar el texto como tal, y solo trabajar con los embeddings. Así ahorraremos memoria. La propia URL nos sirve para saber a qué documento hace mención el embedding en cuestión.

Cabe mencionar que, según dicen en la página del dataset, los embeddings los han creado mediante el modelo de embedding [Cohere Embed V3](https://txt.cohere.com/introducing-embed-v3/), más concrétamente, mediante el modelo [Cohere-embed-multilingual-v3.0](https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0). Esto es importante para saber cuál es la función de embedding que debemos especificar cuando creemos la colección de Chroma.

En definitiva, en nuestra colección guardaremos, para cada documento, el identificador, el embedding, el título como documento (así tendremos mayor flexibilidad en las consultas) y, como metadatos, la URL y el idioma.

Por suspuesto, antes de hablar de colecciones, debemos obtener el cliente. En este caso, trabajaremos en disco (cliente persistente). Así podremos reiniciar el kernel sin perder los datos.


In [3]:
client = chromadb.PersistentClient(path = "./chromadb")
client.heartbeat() # de esta forma comprobamos la conexión

1712408104764643237

Ahora debemos crear la colección. Para ello deberemos también conseguir la función de embedding concreta, puesto que queremos hacer consultas mediante texto y no mediante embeddings. El siguiente código se puede entender revisando los links [uno](https://docs.trychroma.com/embeddings#custom-embedding-functions) y [dos](https://huggingface.co/Cohere/Cohere-embed-multilingual-v3.0).

In [4]:
class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.create_collection(
  name = "wikipedia" , # nombre de la colección
  embedding_function = emb_fn , # función de embedding
  metadata = {"hnsw:space" : "cosine"} # función de distancia entre embeddings
)

Ahora vamos a añadir datos a la colección. Como el dataset es bastante grande, vamos a meterlo poco a poco. Para ello primero creamos una función que cumpla con el cometido.

In [5]:
def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"] ,
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 = n_1 # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

Ya estamos listos para añadir los embeddings.

In [6]:
add_dataset(wikipedia_eu.remove_columns("text") , collection , "eu" , L = 800)

Porcentaje completado: 0.06
Porcentaje completado: 0.12
Porcentaje completado: 0.18
Porcentaje completado: 0.24
Porcentaje completado: 0.30
Porcentaje completado: 0.36
Porcentaje completado: 0.42
Porcentaje completado: 0.48
Porcentaje completado: 0.54
Porcentaje completado: 0.60
Porcentaje completado: 0.66
Porcentaje completado: 0.72
Porcentaje completado: 0.78
Porcentaje completado: 0.84
Porcentaje completado: 0.90
Porcentaje completado: 0.96
Porcentaje completado: 1.02
Porcentaje completado: 1.08
Porcentaje completado: 1.14
Porcentaje completado: 1.21
Porcentaje completado: 1.27
Porcentaje completado: 1.33
Porcentaje completado: 1.39
Porcentaje completado: 1.45
Porcentaje completado: 1.51
Porcentaje completado: 1.57
Porcentaje completado: 1.63
Porcentaje completado: 1.69
Porcentaje completado: 1.75
Porcentaje completado: 1.81
Porcentaje completado: 1.87
Porcentaje completado: 1.93
Porcentaje completado: 1.99
Porcentaje completado: 2.05
Porcentaje completado: 2.11
Porcentaje completad

Comprobamos que todo está correcto. Para ello tenemos las siguientes funciones:

In [ ]:
print("Numero de documentos:" , collection.count())
primero = collection.peek(1) # extraemos el primer elemento
print("ID:" , primero["ids"][0])
print("Embedding:" , str(primero["embeddings"][0][:2])[:-1] , "..." , str(primero["embeddings"][0][-2:])[1:])
print("Metadatos:" , primero["metadatas"][0])
print("Título:" , primero["documents"][0])

Numero de documentos: 1327579
ID: 20231101.eu_1000001_0
Embedding: [0.0100860595703125, -0.0005135536193847656 ... 0.037078857421875, -0.05780029296875]
Metadatos: {'lang': 'eu', 'url': 'https://eu.wikipedia.org/wiki/Monterreyko%20Teknologikoa'}
Título: Monterreyko Teknologikoa


Ya tenemos los documentos en euskara en la base de datos, ahora vamos a hacer lo mismo para los documentos en gallego.

Vamos a aprovechar a reiniciar el kernel, así que tendremos que reescribir el código ya que perderemos las variables.

In [ ]:
import chromadb
from datasets import load_dataset
import cohere

In [8]:
wikipedia_gl = load_dataset("Cohere/wikipedia-2023-11-embed-multilingual-v3", "gl", split="train").remove_columns("text")

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
client = chromadb.PersistentClient(path = "./chromadb")

class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

def add_dataset(ds , collection , idioma , L = 500):
  # L indica el número de documentos a añadir a la vez
  N = ds.num_rows # número total de documentos

  n_0 = 0 # posición del primer elemento a añadir
  while n_0 < N:
    n_1 = min(n_0 + L , N)

    # Nos quedamos con los documentos a añadir y escribimos también el idioma
    ds_0 = ds.select(range(n_0 , n_1)).add_column("lang" , [idioma] * (n_1 - n_0))

    # Añadimos los embeddings, junto a los metadatos y los identificadores
    collection.add(
        embeddings = ds_0["emb"] ,
        documents = ds_0["title"] ,
        metadatas = ds_0.select_columns(["url" , "lang"]).to_list() ,
        ids = ds_0["_id"]
    )

    n_0 = n_1 # actualizamos n_0
    print("Porcentaje completado: %.2f" % (100 * n_0 / N))

add_dataset(wikipedia_gl , collection , "gl" , L = 800)

Porcentaje completado: 0.08
Porcentaje completado: 0.15
Porcentaje completado: 0.23
Porcentaje completado: 0.30
Porcentaje completado: 0.38
Porcentaje completado: 0.45
Porcentaje completado: 0.53
Porcentaje completado: 0.61
Porcentaje completado: 0.68
Porcentaje completado: 0.76
Porcentaje completado: 0.83
Porcentaje completado: 0.91
Porcentaje completado: 0.98
Porcentaje completado: 1.06
Porcentaje completado: 1.14
Porcentaje completado: 1.21
Porcentaje completado: 1.29
Porcentaje completado: 1.36
Porcentaje completado: 1.44
Porcentaje completado: 1.51
Porcentaje completado: 1.59
Porcentaje completado: 1.67
Porcentaje completado: 1.74
Porcentaje completado: 1.82
Porcentaje completado: 1.89
Porcentaje completado: 1.97
Porcentaje completado: 2.04
Porcentaje completado: 2.12
Porcentaje completado: 2.19
Porcentaje completado: 2.27
Porcentaje completado: 2.35
Porcentaje completado: 2.42
Porcentaje completado: 2.50
Porcentaje completado: 2.57
Porcentaje completado: 2.65
Porcentaje completad

In [4]:
collection.count()

2384569

Ya están todos los datos cargados. Vamos a reiniciar de nuevo el kernel.

## Consultas

Ya hemos terminado de cargar documentos. Ahora nos interesa consultar la colección. Trataremos de mostrar las distintas herramientas que tenemos a nuestra disposición.

In [6]:
import chromadb
from datasets import load_dataset
import cohere

In [8]:
client = chromadb.PersistentClient(path = "./chromadb")

class MyEmbeddingFunction(chromadb.EmbeddingFunction):
  def __call__(self, input: chromadb.Documents) -> chromadb.Embeddings:
    co = cohere.Client("") # API key de Cohere
    embeddings = co.embed(
        texts = input ,
        input_type = "search_query" , # solo usaremos el embedding function para las querys
        model = "embed-multilingual-v3.0"
    ).embeddings
    return embeddings

emb_fn = MyEmbeddingFunction()

collection = client.get_collection(
  name = "wikipedia" ,
  embedding_function = emb_fn ,
)

En primer lugar, vamos simplemente a filtrar embeddings como una base de datos al uso. Por ejemplo, vamos a quedarnos con los URL de los documentos en euskera cuyo título contenga la palabra "Lizarra". Para esto tenemos el método `get`.

In [3]:
respuesta = collection.get(
    include = ["metadatas"] , # la URL está en los metadatos
    where = {
        "lang" : "eu" # queremos que el idioma sea euskera
    } ,
    where_document = {"$contains" : "Lizarra"} # queremos que el título contenga "Lizarra"
)
{met["url"] for met in respuesta["metadatas"]} # mostramos un conjunto con las URLs

{'https://eu.wikipedia.org/wiki/Amaia%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Ander%20Lizarralde%20Jimeno',
 'https://eu.wikipedia.org/wiki/Antonio%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Apolonia%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Cris%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Diego%20Lizarrakoa',
 'https://eu.wikipedia.org/wiki/Done%20Mikel%20eliza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Elene%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Eneko%20Lizarralde',
 'https://eu.wikipedia.org/wiki/Foruen%20plaza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Gabriel%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Garazi%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Garestik%20Lizarrara%20%28Donejakue%20bidea%29',
 'https://eu.wikipedia.org/wiki/Gerardo%20Lizarraga',
 'https://eu.wikipedia.org/wiki/Gobernadorearen%20Jauregia%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Hilobi%20Santuaren%20eliza%20%28Lizarra%29',
 'https://eu.wikipedia.org/wiki/Ir

Como vemos, hemos obtenido una ristra de URLs en las que se puede apreciar la palabra "Lizarra" en el título del documento al que identifican.

Pasamos a las consultas. Para comenzar, vamos a hacer una pregunta en euskera. Por ejemplo, le vamos a preguntar "¿Quién es el padre de la informática moderna?". Queremos obtener el título, idioma y URL de los 5 documentos más significativos para esa pregunta.

In [14]:
respuesta = collection.query(
    query_texts = ["Nor da informatika modernoaren gurasoa?"] ,
    n_results = 5 ,
    include = ["documents" , "metadatas"]
)
for i , (met , tit) in enumerate(zip(respuesta["metadatas"][0] , respuesta["documents"][0])):
  sep = "\t" if len(tit) > 5 else "\t\t"
  print(i + 1 , "\tTítulo:" , tit , sep + "Idioma:" , met["lang"] , "\tURL:" , met["url"])

1 	Título: Ekainaren 23 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Ekainaren%2023
2 	Título: 1912 		Idioma: eu 	URL: https://eu.wikipedia.org/wiki/1912
3 	Título: Ekainaren 7 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Ekainaren%207
4 	Título: 1954 		Idioma: eu 	URL: https://eu.wikipedia.org/wiki/1954
5 	Título: Alan Turing 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Alan%20Turing


Lo cierto es que esta respuesta es sorprendente.

Antes de nada, hay que aclarar que "ekaina" es "junio" en euskera. Así que los títulos de los documentos, en el orden en el que nos lo ha dado la consulta son: 23 de junio, 1912, 7 de junio, 1954, Alan Turing.

Lo primero que destaca es "Alan Turing". Precisamente a él se le considera el padre de la informática moderna, por lo que es un gran acierto que nos proponga su página de la Wikipedia. Sin embargo, los cuatro primeros documentos se refieren a años y días concretos... Menuda sorpresa cuando te das cuenta de que Alan Turing nació el 23 de junio de 1912 y falleció el 7 de junio de 1954.

En resumen:
- Hemos escrito una cadena de caracteres. Esta cadena tiene significado en euskera: estamos preguntando por una persona concreta.
- No hemos puesto ningún filtro a los documentos.
- Los cinco documentos que nos propone la consulta están en euskera. Uno nos señala a la persona por la que preguntábamos y los otros cuatro nos indican su fecha de nacimiento y muerte.

La verdad que no nos esperábamos estos resultados, parecen demasiado buenos.

Vamos ahora a hacer una pregunta en gallego. En este caso vamos a preguntar "¿Qué escritora gallega destacó por su poesía romántica en el siglo XIX?".

In [17]:
respuesta = collection.query(
    query_texts = ["Que escritora galega destacou pola súa poesía romántica no século XIX?"] ,
    n_results = 5 ,
    include = ["documents" , "metadatas"]
)
for i , (met , tit) in enumerate(zip(respuesta["metadatas"][0] , respuesta["documents"][0])):
  sep = "\t" if len(tit) > 13 else "\t\t\t"
  print(i + 1 , "\tTítulo:" , tit , sep + "Idioma:" , met["lang"] , "\tURL:" , met["url"])

1 	Título: Pintura do romanticismo 	Idioma: gl 	URL: https://gl.wikipedia.org/wiki/Pintura%20do%20romanticismo
2 	Título: Historia da lingua galega 	Idioma: gl 	URL: https://gl.wikipedia.org/wiki/Historia%20da%20lingua%20galega
3 	Título: Literatura galega do século XX 	Idioma: gl 	URL: https://gl.wikipedia.org/wiki/Literatura%20galega%20do%20s%C3%A9culo%20XX
4 	Título: Historia da lingua galega 	Idioma: gl 	URL: https://gl.wikipedia.org/wiki/Historia%20da%20lingua%20galega
5 	Título: Precursores 			Idioma: gl 	URL: https://gl.wikipedia.org/wiki/Precursores


Los resultados siguen siendo siendo impresionantes, aunque quizás no tan sorprendentes como los anteriores. Obtenemos 4 documentos (2 están repetidos) en gallego, unos sobre la pintura del romanticismo, dos sobre la lengua gallega (historia y literatura del siglo XX) y otro cuyo título es "Precursores". Si revisamos los cuatro documentos, veremos que en los cuatro se menciona a Rosalía de Castro.

Ahora, simplemente por mostrar filtros en la consulta, vamos a preguntar por el rey Sancho el sabio, solo considerando documentos en euskera y cuyo título contenga los caracteres "historia" o "Historia".

In [7]:
respuesta = collection.query(
    query_texts = ["Antso Azkarra"] ,
    n_results = 10 ,
    where = {"lang" : "eu"} ,
    where_document = {
        # De esta forma se escribe el OR lógico
        "$or" : [
            {"$contains" : "historia"} ,
            {"$contains" : "Historia"}
        ]
    }
)
for i , (met , tit) in enumerate(zip(respuesta["metadatas"][0] , respuesta["documents"][0])):
  sep = "\t" if len(tit) > 27 else "\t\t" if len(tit) > 22 else "\t\t\t"
  print(i + 1 , "\tTítulo:" , tit , sep + "Idioma:" , met["lang"] , "\tURL:" , met["url"])

1 	Título: Euskal Herriko historiaren kronologia 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Euskal%20Herriko%20historiaren%20kronologia
2 	Título: Gipuzkoako historia 			Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Gipuzkoako%20historia
3 	Título: Euskal Herriko historia 		Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Euskal%20Herriko%20historia
4 	Título: Euskal Herriko historiaren kronologia 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Euskal%20Herriko%20historiaren%20kronologia
5 	Título: Nafarroako historia 			Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Nafarroako%20historia
6 	Título: Donostiako bertsolaritzaren historia 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Donostiako%20bertsolaritzaren%20historia
7 	Título: Bilboko Athletic Klubaren historia 	Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Bilboko%20Athletic%20Klubaren%20historia
8 	Título: Donostiako historia 			Idioma: eu 	URL: https://eu.wikipedia.org/wiki/Donostiako%20historia
9 	Título: Filosofiaren h

# Representando los documentos

Lo que vamos a tratar de hacer ahora es representar gráficamente los documentos. Para ello, reduciremos los embeddings a dos dimensiones mediante las técnicas PCA y TSNE.

En primer lugar, vamos a extraer 512 documentos en euskera y otros 512 en gallego. Realmente, tal y como hemos podido comprobar en alguna consulta, hay documentos repetidos puesto que son versiones distintas. Esto va a ensuciar un poco la gráfica, pero tampoco nos queremos complicar más.

In [12]:
eu = collection.get(where = {"lang" : "eu"} , limit = 512 , include = ["embeddings" , "documents"])
gl = collection.get(where = {"lang" : "gl"} , limit = 512 , include = ["embeddings" , "documents"])

emb = eu["embeddings"] + gl["embeddings"]
tit = (list(map(lambda titulo : "EU " + titulo , eu["documents"])) +
       list(map(lambda titulo : "GL " + titulo , gl["documents"])))

Ya tenemos una lista con los 1024 embeddings y otra con los títulos junto con un identificador del idioma.

Ahora, aprovechando la asignatura de aprendizaje profundo, vamos a definir una función para representar los vectores bidimensionales junto con las etiquetas, de forma que al pasar el ratón por los puntos podamos ver las etiquetas.

In [1]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
  """ draws an interactive plot for data points with auxilirary info on hover """
  if isinstance(color, str): color = [color] * len(x)
  data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

  fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
  fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

  fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
  if show: pl.show(fig)
  return fig

Ya solo falta reducir las dimensiones de los embeddings. Primero reduciremos de 1024 a 30 dimensiones mediante la técnica PCA. A continuación, reduciremos a 2 dimensiones mediante TSNE. Por último, estandarizaremos los datos. Tras ello, representaremos los puntos.

In [18]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

vector = StandardScaler().fit_transform(
    TSNE().fit_transform(PCA(n_components = 30).fit_transform(emb))
)

draw_vectors(vector[: , 0] , vector[: , 1] ,
             color = 'green' , token = tit)

figure(id='p1053', ...)

Podemos observar que hay una separación bastante clara entre documentos en euskera y en gallego, en función del valor de la primera componente. La mayoría de los "puntos gordos" son distintas versiones del mismo documento, aunque hay concentraciones de documentos parecidos, por ejemplo, hay muchos documentos sobre universidades escritos en euskera en la misma zona.

Por supuesto, la gráfica sería mucho más interesante teniendo en cuenta todos los documentos, pero eso requeriría muchísima potencia computacional.

## Conclusiones

Respecto a los resultados obtenidos, la verdad que han sido sorprendentes. Por supuesto, se debe sobre todo a la función de embedding y, probablemente, a que hemos elegido la similitud del coseno como función de distancia. En cualquier caso, resulta muy interesante que está potencia de búsqueda pueda estar implementada directamente en la baase de datos.

Respecto a los aspectos técnicos, cabe decir que añadir los datos ha sido un proceso bastante lento y tedioso. Suponemos que se debe sobre todo a la escasa memoria RAM que proporciona Google Colab, además de los riesgos de que se te vaya la sesión y demás problemas que ya conocemos.

Nos parece destacable que Chroma trabaja en memoria RAM, en el sentido de que, pese a conectarnos a un cliente persistente y que los datos se guarden en disco, cada vez que iba a realizar cualquier operación (añadir datos o consultar) volcaba todo el contenido a la memoria RAM.